In [1]:
import pandas as pd 
from scipy.stats import chi2_contingency
from scipy.stats.contingency import association
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [2]:
df2 = pd.read_csv("../data/clean/df_processed.csv")
df2.drop(['Unnamed: 0'],inplace=True,axis=1)

In [5]:
import pandas as pd
from sklearn.utils import resample

# Group df2 by the values of the 'readmitted' column
groups = df2.groupby('readmitted')

# Sample equally from each group to create the balanced DataFrame
balanced_dfs = []
for _, group in groups:
    balanced_sample = resample(group, n_samples=len(df2) // 2 // len(groups), random_state=42)
    balanced_dfs.append(balanced_sample)

# Concatenate the sampled DataFrames
balanced_df = pd.concat(balanced_dfs)

# Shuffle the concatenated DataFrame
df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)

#df represents a DataFrame with an equal distribution of 'readmitted' values
df.sample(frac=1, random_state=42).reset_index(drop=True)

 

,race,gender,age,weight,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,change,diabetesMed,readmitted
0,caucasian,male,[70-80),[100-125),3,63,0,14,0,1,...,no,no,no,no,no,no,no,no,no,>30
1,caucasian,male,[50-60),[100-125),3,68,3,21,1,0,...,no,no,no,no,no,no,no,no,no,>30
2,caucasian,female,[60-70),[50-75),5,33,1,8,0,0,...,no,no,no,no,no,no,no,no,yes,no
3,caucasian,male,[60-70),[125-150),4,36,2,21,0,0,...,no,no,no,no,no,no,no,no,no,no
4,caucasian,male,[80-90),[75-100),3,44,1,11,0,0,...,no,no,no,no,no,no,no,no,no,>30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,caucasian,female,[90-100),[50-75),4,70,0,17,0,0,...,no,no,steady,no,no,no,no,no,yes,<30
515,caucasian,male,[60-70),[100-125),4,42,1,23,0,0,...,no,no,no,no,no,no,steady,no,yes,>30
516,caucasian,female,[30-40),[50-75),5,39,1,6,1,0,...,no,no,no,no,no,no,steady,no,yes,<30
517,caucasian,female,[70-80),[25-50),4,72,3,13,2,0,...,no,no,no,no,no,no,steady,no,yes,no


In [6]:
df

,race,gender,age,weight,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,...,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,acarbose,insulin,change,diabetesMed,readmitted
0,caucasian,male,[50-60),[100-125),1,69,0,10,0,0,...,no,no,steady,no,no,no,no,no,yes,>30
1,africanamerican,female,[60-70),[50-75),6,56,3,13,1,3,...,no,no,no,no,no,no,no,no,no,<30
2,caucasian,female,[50-60),[100-125),2,36,2,18,1,0,...,no,no,no,no,no,no,no,no,yes,<30
3,caucasian,male,[60-70),[100-125),2,24,2,22,0,0,...,no,steady,no,no,steady,no,no,ch,yes,<30
4,caucasian,female,[60-70),[50-75),3,62,0,12,1,0,...,no,no,no,no,no,no,no,no,yes,<30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
514,caucasian,female,[50-60),[100-125),3,53,1,25,2,0,...,no,no,no,steady,no,no,no,ch,yes,<30
515,caucasian,female,[80-90),[75-100),3,18,2,6,0,0,...,no,no,no,no,no,no,no,no,no,<30
516,caucasian,male,[70-80),[75-100),3,68,6,21,0,0,...,no,no,steady,steady,no,no,no,ch,yes,>30
517,caucasian,male,[70-80),[125-150),3,10,1,12,0,0,...,no,no,no,no,no,no,steady,ch,yes,no


In [7]:
features = df.drop(['readmitted'],axis=1)
target = df[['readmitted']]

In [8]:
df_cat = df.select_dtypes('object')
df_num = df.select_dtypes('number')


## 1st Try with all data though distribution of target values are not equal

In [9]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.2, random_state=3)

In [10]:
categorical_column_values = [ X_train[col].unique() for col in X_train.select_dtypes('object').columns ]

In [11]:
encoder = OneHotEncoder(drop='first', sparse_output=False)

In [14]:
encoder.fit(X_train_cat)

OneHotEncoder(drop='first', sparse_output=False)

In [15]:
X_train_cat = X_train.select_dtypes('object')
X_test_cat  = X_test.select_dtypes('object')
#
X_train_num = X_train.select_dtypes('number')
X_test_num  = X_test.select_dtypes('number')

In [16]:
X_train_cat_encoded_np = encoder.transform(X_train_cat)
X_test_cat_encoded_np  = encoder.transform(X_test_cat)


In [17]:
X_train_cat_encoded_df = pd.DataFrame(X_train_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_train.index)
X_test_cat_encoded_df = pd.DataFrame(X_test_cat_encoded_np, columns=encoder.get_feature_names_out(), index=X_test.index)

In [18]:
X_test_cat_encoded_df

,race_asian,race_caucasian,race_other,gender_male,age_[30-40),age_[40-50),age_[50-60),age_[60-70),age_[70-80),age_[80-90),...,pioglitazone_no,pioglitazone_steady,pioglitazone_up,rosiglitazone_steady,acarbose_steady,insulin_no,insulin_steady,insulin_up,change_no,diabetesMed_yes
16,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
57,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
240,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
353,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
111,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
185,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
159,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
389,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


In [19]:
X_train = pd.concat([X_train_num, X_train_cat_encoded_df], axis=1)
X_test = pd.concat([X_test_num, X_test_cat_encoded_df], axis=1)

In [20]:
X_train

,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,race_asian,race_caucasian,...,pioglitazone_no,pioglitazone_steady,pioglitazone_up,rosiglitazone_steady,acarbose_steady,insulin_no,insulin_steady,insulin_up,change_no,diabetesMed_yes
273,3,38,2,15,1,2,9,9,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
237,2,31,1,14,0,0,0,7,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
506,1,56,0,6,0,2,0,9,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
121,3,55,4,14,0,0,0,6,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
508,3,56,0,8,0,1,0,5,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
480,2,5,1,13,0,0,0,5,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
422,6,58,2,17,3,3,2,9,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0
119,1,69,1,20,0,0,0,9,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0
256,3,18,4,10,0,0,0,6,0.0,1.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0


In [21]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)

MinMaxScaler()

In [22]:
X_train_norm_np = normalizer.transform(X_train)
X_test_norm_np = normalizer.transform(X_test)

In [23]:
X_train_norm_df = pd.DataFrame(X_train_norm_np, columns=X_train.columns, index=X_train.index)
X_test_norm_df  = pd.DataFrame(X_test_norm_np,  columns=X_test.columns,  index=X_test.index)

In [24]:
knn = KNeighborsClassifier(n_neighbors=3)  
knn.fit(X_train_norm_df, y_train)
# Save the model with pickle

/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/neighbors/_classification.py:238: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


KNeighborsClassifier(n_neighbors=3)

In [25]:
y_pred = knn.predict(X_test_norm_df)

In [26]:
accuracy = knn.score(X_test_norm_df, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.5384615384615384


In [27]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)
dt_predictions = dt_classifier.predict(X_test)
dt_accuracy = accuracy_score(y_test, dt_predictions)
print("Decision Tree Accuracy:", dt_accuracy)
rf_predictions = rf_classifier.predict(X_test)
rf_accuracy = accuracy_score(y_test, rf_predictions)
print("Random Forest Accuracy:", rf_accuracy)


/Users/og/Desktop/IronHackDA/7-Final_Project/SugarSync/sugarsync/lib/python3.11/site-packages/sklearn/base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


Decision Tree Accuracy: 0.75
Random Forest Accuracy: 0.7596153846153846


In [ ]:
rf_predictions = rf_classifier.predict(X_test_full)
rf_accuracy = accuracy_score(y_test, rf_predictions)

In [ ]:
rf_accuracy